In [1]:
import numpy as np
import scipy.io as sio
import os
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM
from tensorflow.keras.models import Sequential, Model

import keras
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
from keras import backend as K
import time
from sklearn.model_selection import StratifiedKFold

In [20]:
num_classes = 2
batch_size = 128
img_rows, img_cols, num_chan = 8, 9, 4
flag = 'a'
t = 6

acc_list = []
std_list = []
all_acc = []

short_name = ['01', '02', '03', '04', '05', '06', '07', '08', '09',
              '10', '11', '12', '13', '14', '15', '16', '17', '18',
              '19', '20', '21', '22', '23', '24', '25', '26', '27',
              '28', '29', '30', '31', '32']

# 45次实验分别进行10倍交叉验证，取平均
i = 0
dataset_dir = "/home/hyojinju/4D-CRNN/DEAP/with_base_0.5/" #"/home/kaka/Desktop/sfy_file/eeg_emotion/nonCrossSubject/data/DEAP/with_base_0.5/"

file_path = os.path.join(dataset_dir, 'DE_s' + short_name[i] + '.dat')
K.clear_session()
start = time.time()
print("\nprocessing: ", short_name[i], "......")
file = sio.loadmat(file_path)
data = file['data']
y_v = file['valence_labels'][0]
y_a = file['arousal_labels'][0]
print("\nY shape initial", y_v.shape)
y_v = to_categorical(y_v, num_classes)
y_a = to_categorical(y_a, num_classes)
print("\nY shape tocategorical", y_v.shape)

print("\nData shape:",data.shape)
one_falx = data.transpose([0, 2, 3, 1])
# one_falx = one_falx[:,:,:,2:4]
print("\nTranspose:", one_falx.shape)
one_falx = one_falx.reshape((-1, t, img_rows, img_cols, num_chan))
print("\nReshape:", one_falx.shape)
one_y_v = np.empty([0,2])
print("\nY shape", one_y_v.shape)
one_y_a = np.empty([0,2])
for j in range(int(len(y_a)//t)):
    one_y_v = np.vstack((one_y_v, y_v[j*t]))
    one_y_a = np.vstack((one_y_a, y_a[j*t]))
print("\nY shape segment", one_y_v.shape)

# print(one_y_v.shape)
# print(one_y_a.shape)
# print(one_falx.shape)

if flag=='v':
    one_y = one_y_v
else:
    one_y = one_y_a



processing:  01 ......

Y shape initial (4800,)

Y shape tocategorical (4800, 2)

Data shape: (4800, 4, 8, 9)

Transpose: (4800, 8, 9, 4)

Reshape: (800, 6, 8, 9, 4)

Y shape (0, 2)

Y shape segment (800, 2)


In [34]:
data = np.empty((0, 4, 8 ,9))
y_v = np.empty((0,))
y_a = np.empty((0,))
# 45次实验分别进行10倍交叉验证，取平均
dataset_dir = "/home/hyojinju/4D-CRNN/DEAP/with_base_0.5/" #"/home/kaka/Desktop/sfy_file/eeg_emotion/nonCrossSubject/data/DEAP/with_base_0.5/"
for i in range(len(short_name)):
    # K.clear_session()
    # start = time.time()
    # print("\nprocessing: ", short_name[i], "......")
    file_path = os.path.join(dataset_dir, 'DE_s'+short_name[i]+'.dat')
    file = sio.loadmat(file_path)
    data = np.vstack((data, file['data']))
    y_v = np.concatenate((y_v, file['valence_labels'][0]), axis=0)
    y_a = np.concatenate((y_a, file['arousal_labels'][0]), axis=0)

    # y_v.append(file['valence_labels'][0])
    # y_a.append(file['arousal_labels'][0])
    #= np.vstack((y_a, file['arousal_labels'][0]))

# print(data.shape)
# print(np.array(y_v).shape)
# print(np.array(y_a).shape)
print(y_a[:1400])

[1. 1. 1. ... 0. 0. 0.]


In [26]:
splits = 5

tx = range(10)
ty = [0,1,0,1,0,1,0,1,0,1] #[0] * 5 + [1] * 5

from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn import datasets

stratKfold = StratifiedKFold(n_splits=splits, shuffle=False, random_state=None)#, random_state=42)
shufflesplit = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)#, random_state=42)
# shufflesplit = StratifiedShuffleSplit(n_splits=splits, random_state=42, test_size=2)

print("stratKFold")
for train_index, test_index in stratKfold.split(tx, ty):
    print("TRAIN:", train_index, "TEST:", test_index)

print("Shuffle Split")
for train_index, test_index in shufflesplit.split(tx, ty):
    print("TRAIN:", train_index, "TEST:", test_index)


stratKFold
TRAIN: [2 3 4 5 6 7 8 9] TEST: [0 1]
TRAIN: [0 1 4 5 6 7 8 9] TEST: [2 3]
TRAIN: [0 1 2 3 6 7 8 9] TEST: [4 5]
TRAIN: [0 1 2 3 4 5 8 9] TEST: [6 7]
TRAIN: [0 1 2 3 4 5 6 7] TEST: [8 9]
Shuffle Split
TRAIN: [0 1 2 3 4 5 8 9] TEST: [6 7]
TRAIN: [1 2 4 5 6 7 8 9] TEST: [0 3]
TRAIN: [0 1 2 3 6 7 8 9] TEST: [4 5]
TRAIN: [0 2 3 4 5 6 7 9] TEST: [1 8]
TRAIN: [0 1 3 4 5 6 7 8] TEST: [2 9]
